# ECON622: Problem Set 1

Jesse Perla, UBC

## Student Name/Number: Wan-Chiao Chang/17065012

## Question 1

Follow instructions in [Python Environment
Setup](../pages/python_setup.qmd) to get your python environment setup.

Some imported packages which will tell you if you have a successful
installation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import torch
import jax
import jax.numpy as jnp
from jax import grad, hessian

Mess around with your setup until the following work

In [2]:
x = torch.zeros((2, 4))
print(x)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [3]:
x = jnp.zeros((2, 3))
print(x)

[[0. 0. 0.]
 [0. 0. 0.]]


No need to get GPUs/etc. working. If you have issues on windows, see
[JAX
Installation](https://jax.readthedocs.io/en/latest/installation.html#cpu)
and [Pytorch Installation](https://pytorch.org/get-started/locally/)
instructions

## Question 2

This section just summarizes some reading on how JAX works, and asks you
to come up with some demonstrations of its functionality. Read:

-   [JAX
    Quickstart](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html)
-   [JAX
    Basics](https://jax.readthedocs.io/en/latest/jax-101/01-jax-basics.html)
-   [JAX
    Jit](https://jax.readthedocs.io/en/latest/jax-101/02-jitting.html)
-   [JAX
    Vectorization](https://jax.readthedocs.io/en/latest/jax-101/03-vectorization.html)
-   [JAX
    AD](https://jax.readthedocs.io/en/latest/jax-101/04-advanced-autodiff.html)

### Question 2.1

Write a function which takes a normally distributed random number and
calculates the standard deviation manually (i.e. calculate the mean,
second moment, etc. manually and then form the standard deviation
yourself in the function)

Compile the function using the `jit` and the decorator `@jit`

In [11]:
@jax.jit
def manual_std(x):
    """
    std = sqrt( (1/n) * sum_i (x_i - mean)^2 )
    """
    n = x.shape[0]
    mean = jnp.sum(x) / n
    var = jnp.sum((x - mean) ** 2) / n
    return jnp.sqrt(var)

In [14]:
key = jax.random.key(622)
x = jax.random.normal(key, (1000000,))
s = manual_std(x)
print(s)

1.0005516


### Question 2.2

Now take that function and generate a matrix of random normals and use
`vmap` to calculate the standard deviation across rows, then across
columns. Compare to `jnp.std(x, axis=0)` and `jnp.std(x, axis=1)`

In [19]:
# generate a matrix of random normals
key = jax.random.key(622)
key, subkey = jax.random.split(key)
X = jax.random.normal(subkey, shape=(1000, 500))

# Standard deviation across rows
row_std_vmap = jax.vmap(manual_std, in_axes=0)
row_std = row_std_vmap(X)
row_std_jnp = jnp.std(X, axis=1)

# Standard deviation across columns
col_std_vmap = jax.vmap(manual_std, in_axes=1)
col_std = col_std_vmap(X)
col_std_jnp = jnp.std(X, axis=0)

# Compare to `jnp.std(x, axis=0)` and `jnp.std(x, axis=1)` 
print(jnp.max(jnp.abs(row_std - row_std_jnp)))
print(jnp.max(jnp.abs(col_std - col_std_jnp)))

1.1920929e-07
1.1920929e-07


### Question 2.3

Take the following code, and find the hessian of `f` at `x`

In [ ]:
# Define a multivariate function
def f(x):
    return jnp.sum(x ** 2)

# Define a point at which to evaluate the function
x = jnp.array([1.0, 2.0, 3.0])

# Function that computes the Hessian
H_f = jax.hessian(f)

In [24]:
# Evaluate the Hessian at point x
H_at_x = H_f(x)
print(H_at_x)

[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 2.]]
